# Resynchronizing Jittery AES Power Traces

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

What happens if things aren't as clean as we made them out to be? We can use preprocessing modules!

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
num_traces = 50
CHECK_CORR = False

## Capturing Jittery Traces

### Rebuilding New Firmware

In file `chipwhisperer/hardware/victims/firmware/simpleserial-aes/simpleserial-aes.c` find this:

```c
uint8_t get_pt(uint8_t* pt)
{
	trigger_high();
	aes_indep_enc(pt); /* encrypting the data block */
	trigger_low();
	simpleserial_put('r', 16, pt);
	return 0x00;
}
```

and add some random delay:

```c
uint8_t get_pt(uint8_t* pt)
{
	trigger_high();
       for(volatile uint8_t k = 0; k < (*pt & 0x0F); k++);
	aes_indep_enc(pt); /* encrypting the data block */
	trigger_low();
	simpleserial_put('r', 16, pt);
	return 0x00;
}
```

This deterministic delay is NOT a good countermeasure, but is much easier to write in a single line since we don't have a CSPRNG linked in. We'll break the jitter without relying on the deterministic aspect though, so our attack would work against a better jitter source.

**Be sure to remove this function afterwards so you don't break your code!**

We can build the code (change the platform as needed), and confirm the output of the following works as you expect:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2 EXTRA_OPTS=ADD_JITTER

### Setup

Now let's go ahead. We'll have to program the file we built, so be sure to confirm we are using the right file!

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
import os, time

fw_path = '../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)

modtime = os.path.getmtime(fw_path)
print("File build time: {:s} (built {:.2f} mins ago)".format(str(time.ctime(modtime)), (time.time() - modtime)/60.0))

In [ ]:
cw.program_target(scope, prog, fw_path)

In addition, before we capture our traces, we'll need to create a ChipWhipserer project, since that's what Analyzer expects for an input:

In [ ]:
project = cw.create_project("projects/jittertime", overwrite = True)

### Capturing Traces

Below you can see the capture loop. The main body of the loop loads some new plaintext, arms the scope, sends the key and plaintext, then finally records and our new trace into our trace class. We'll also keep track of our keys manually for checking our answer later.

In [ ]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()


for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)

In [ ]:
#Save project file
project.save()

We're now done with the ChipWhisperer hardware, so we should disconnect from the scope and target:

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Analysis

To fix the jitter, we'll need to add our traces to a preprocessing module. We can feed our project right into `cwa.cpa()`, but we could also add pre-processing inbetween (more about this later).

In [ ]:
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa
project = cw.open_project("projects/jittertime")

This time we're going to do a few things. First we will get the traces, and plot a few of them as-is. You can adjust the traces plotted by adjusting the `range(10)`. For example `range(1)` plots the first trace.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Dark2_5 as palette
import itertools  

output_notebook()
p = figure(sizing_mode='scale_width', plot_height=300)

# create a color iterator
colors = itertools.cycle(palette)  

x_range = range(0, len(project.waves[0]))
for i, color in zip(range(5), colors): #Adjust range(n) to plot certain traces
    p.line(x_range, project.waves[i], color=color)
show(p)

So how do we fix that? To begin with, you should plot only a single trace to make your life more clear. You'll need to figure out a very unique area. For example see the following figure showing a single plot. In this example the location of         
**A** is unique, but B would have many matches within that same trace, even nearby:
![Resync example trace](img/resync_goodbad.png)

We will specify two items:
- A window with the "unique" area defined.
- How far we will shift the window (+/- points) to search for the best match.

You can use the following code to define the `target_window` and `max_shift`. Try a few values until you find something that works. Also try some poor example, and also try plotting more traces to confirm your match is working in real life.

In [ ]:
resync_traces = cwa.preprocessing.ResyncSAD(project)
resync_traces.ref_trace = 0

if PLATFORM == "CWNANO":
    #Define a target window here. 500,900 for example is good based on above. But try some different values.
    resync_traces.target_window = (300, 700)

    # Define max_shift. Must not cause target_window to go outside of valid data. Try 16-600 range. Ideal value varies with how
    # much jitter is in original data. 
    resync_traces.max_shift = 300
elif PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    #Define a target window here. 500,900 for example is good based on above. But try some different values.
    resync_traces.target_window = (1000, 1400)

    # Define max_shift. Must not cause target_window to go outside of valid data. Try 16-600 range. Ideal value varies with how
    # much jitter is in original data. 
    resync_traces.max_shift = 1000
elif PLATFORM=="CWLITEARM":
    #Define a target window here. 500,900 for example is good based on above. But try some different values.
    resync_traces.target_window = (700, 1500)

    # Define max_shift. Must not cause target_window to go outside of valid data. Try 16-600 range. Ideal value varies with how
    # much jitter is in original data. 
    resync_traces.max_shift = 700

#Uses objects from previous cells (plotting etc), so 
output_notebook()
p = figure()
new_proj = resync_traces.preprocess()

for i, color in zip(range(0, 5), colors):
    p.line(x_range, new_proj.waves[i], color=color)
show(p)

If this all works - let's just continue the attack! Do so as below:

In [ ]:
leak_model = cwa.leakage_models.sbox_output
attack = cwa.cpa(new_proj, leak_model)

And then actually run it:

In [ ]:
cb = cwa.get_jupyter_callback(attack)
attack_results = attack.run(cb, 10)

You *should* see the PGE reach 0 for each byte. If not, you might need to adjust the SAD resync. You could also need to increase the length of the sample capture for example. You may notice that it starts working OK and then fails, due to later traces become unsychronized.

### Plotting Correlation Output

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

attack_results = attack.results
plot_data = cwa.analyzer_plots(attack_results)
bnum = 0

ret = plot_data.output_vs_time(bnum)

output_notebook()
p = figure()
p.line(ret[0], ret[2], line_color='green')
p.line(ret[0], ret[3], line_color='green')

p.line(ret[0], ret[1], line_color='red')
show(p)

You should see a graph of red and green in time (samples). In red is the correlation of the correct subkey for the first byte, while the rest are in green.

You should see two or three distinctive red spikes. The first is the spot where the sbox lookup for the subkey we guessed actually happens (the later ones are from later steps in the AES operation).

What about the rest of the bytes in the key? We can get and plot that easily as well:

In [ ]:
rets = []
for i in range(0, 16):
    rets.append(plot_data.output_vs_time(i))

p = figure()
for ret in rets:
    p.line(ret[0], ret[2], line_color='green')
    p.line(ret[0], ret[3], line_color='green')
    
for ret in rets:
    p.line(ret[0], ret[1], line_color='red')

show(p)

## Conclusion

Awesome! You should have now completed a resynchronization of power traces. This is a very useful tool, and you can see how making a simple class could extend this work.

## Tests

In [ ]:
key = list(project.keys[0])
recv_key = [kguess[0][0] for kguess in attack_results.find_maximums()]
assert key == recv_key, "Failed to recover encryption key\nGot: {}\nExpected: {}".format(recv_key, key)

In [ ]:
assert (attack_results.pge == [0]*16), "PGE for some bytes not zero: {}".format(attack_results.pge)

In [ ]:
if CHECK_CORR:
    max_corrs = [kguess[0][2] for kguess in attack_results.find_maximums()]
    assert (np.all([corr > 0.75 for corr in max_corrs])), "Low correlation in attack (corr <= 0.75): {}".format(max_corrs)